# Artificial Inteligence
## Maria Inês Pinto - 904644

# Introduction

This project aims to create a robust handwritten digit recognition system through the use of neural networks. The focal point of our effort is the MNIST database, a rich repository with 60,000 images of handwritten numbers ranging from 0 to 9. Each image is intricately composed of 28x28 pixels, presenting a diverse and challenging dataset. Our strategy involves developing and training a neural network model on this extensive database, culminating in a comprehensive evaluation to assess its effectiveness in recognizing and classifying handwritten digits.
It will be possible for the user to test our ANN by uploading 28x28 images with digits or writing using the touchpad.

## Imports
Three essential Python libraries, **NumPy**, **Matplotlib** and **TensorFlow**, are integral to this project for distinct reasons. **NumPy** is indispensable for **performing efficient and high-performance mathematical operations**, providing an array-oriented computing framework. It serves as a fundamental tool for handling the numerical aspects of our neural network, facilitating matrix manipulations, and supporting the underlying computations crucial for training and evaluation.

**Matplotlib**, on the other hand, is crucial for **visually representing data through plots and charts**. In the context of this project, Matplotlib will be instrumental in illustrating key metrics, visualizing the training process, and showcasing the performance of our handwritten digit recognition system. Graphical representations generated by Matplotlib will offer insights into the effectiveness and learning dynamics of our neural network.

**TensorFlow** provides a **high-level API for building and training neural networks**, efficient computation on CPUs and GPUs, pre-built functions and layers, utility functions for datasets, model saving/loading, a vibrant community, and integration with other ML tools. It simplifies the development process and enables efficient training and deployment of deep learning models, making it a cornerstone for tasks like handwritten digit recognition.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Importing MNIST Data
We download and load MNIST data from Keras

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

We have 60000 training image with a size of 28 x 28

In [ ]:
print('x_train: ' + str(x_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_train: ' + str(y_train.shape))
print('y_test: ' + str(y_test.shape))

## Visualization
This code aims to offer a practical and informative visualization of images belonging to the MNIST dataset. The set contains handwritten numeric representations of digits 0 to 9.

The specific image at index 7849 is initially displayed, providing a detailed look at how one of the examples in the dataset appears visually, in this case the number 7.

Additionally, the code creates a subplot that displays the first instances of each digit from 0 to 9. Each subplot is accompanied by the label corresponding to the digit, providing a quick and efficient overview of the visual characteristics associated with each number. This is particularly useful for familiarization and visual inspection of the dataset, essential in handwritten digit recognition tasks.

In [ ]:
image_index = 7849
print(y_train[image_index])
plt.imshow(x_train[image_index], cmap = 'Greys')

In [ ]:
f, ax = plt.subplots(1, 10, figsize = (20,20))

for i in range(0,10):
  temp = x_train[y_train == i][0]
  ax[i].imshow(temp, cmap='gray')
  ax[i].set_title("Label: {}".format(i), fontsize = 15)

## Normalization
tf.keras.utils.normalize function scales pixel values to a standard range (typically 0 to 1). This practice ensures numerical stability, accelerates convergence during training, and promotes better generalization by making the neural network less sensitive to input variations. Normalization enhances the model's performance and stability, contributing to efficient learning and improved adaptability to new, unseen data.

In [ ]:
x_train = tf.keras.utils.normalize(x_train, axis = 1)
x_test = tf.keras.utils.normalize(x_test, axis = 1)

# Model Creation
To construct our neural network architecture, we will leverage the **Keras library**, a high-level neural networks API. Keras simplifies the implementation of complex neural network models by providing user-friendly interfaces. Specifically, we will make use of the **Sequential model** and **Dense layers** for defining the architecture of our neural network. This combination of libraries empowers us to seamlessly integrate mathematical operations, visualization capabilities, and a user-friendly neural network framework for an effective and comprehensive approach to handwritten digit recognition.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28, 28)))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Train the Model

In [ ]:
model.fit(x_train, y_train, epochs = 100)

model.save('digits.model')

Instead of always fitting the model, we comment the lines before and simply load the saved model.
This metrics tell us how accurate our model is.
We wanna have a **low loss** and a **high accuracy**

In [ ]:
model = tf.keras.models.load_model('digits.model')
loss, accuracy = model.evaluate(x_test, y_test)
print (loss)
print (accuracy)

## Create Flask App
We need to create a flask app to handle user requests from the front end and render html using template engine. Right know we are inside the folder **"app"** in **"main.py"**.

In [ ]:
import os
from flask import Flask, render_template, request, jsonify
import numpy as np
from tensorflow import keras
import cv2
import base64

# App and model initializer
app = Flask(__name__)
title = 'Hand Digit Recognizer'

# Loading prebuilt AI
model = keras.models.load_model('app/digits.model')

Since we have two ways to test our app, we have two html pages. 
Therefore, the home() function is associated with the root URL and renders the 'home.html' template, while the drawing() function is associated with the "/drawing" URL for GET requests and renders the 'drawing.html' template.

In [ ]:
# GET method
@app.route('/')
def home():
    return render_template('home.html', title=title)

@app.route('/drawing', methods=['GET'])
def drawing():
    return render_template('drawing.html', title=title)

### Prediction from file

The provided Flask route handles POST requests to the root URL ('/'), expecting an uploaded file named 'file.' Upon receiving a file, the code reads its content and processes it as a grayscale image. The image is resized to 28x28 pixels and formatted for input into a pre-trained neural network model. The model predicts the image's class, and the result is printed. The prediction, along with the application title, is then passed to the 'home.html' template for rendering a response. If successful, the predicted class is displayed; otherwise, any exception is caught and displayed as an error message.

In [ ]:
@app.route('/', methods=['POST'])
def result():
    print('Post request recieved')
    file_str = request.files['file'].read()
    file_np = np.fromstring(file_str, np.uint8)
    print(f'File recieved : {file_np.shape}')

    file_np = cv2.resize(file_np,(28,28))
    file_np = np.expand_dims(file_np, axis=0)

    try:
        prediction = np.argmax(model.predict(file_np))
        print(f"Prediction : {str(prediction)}")
        return render_template('home.html', title=title, response=str(prediction), success=True)
    except Exception as e:
        return render_template('home.html', title=title, response=str(e))   



### Prediction from drawing
After receiving base64 data from front end, we need to convert the image from 3 channel to 1 channel (at line 30) because we train the model on one channel images. The image we received from front end has 3 channel (RBG) with the shape (280, 280, 3) and we convert to (280, 280).

We need to resize the image to 28 x 28 (at line 33). We need resize the image because our original size was 280 x 280. You can check on canvas tag with width and height of 280 (at line 43). Remember our model was trained using 28 x 28 image, so if we want to predict any images, we need to resize into 28 x 28. Also expand the array dimension from (28, 28) to (1, 28, 28) because the number 1 means we have 1 images with the size 28 x 28.



In [ ]:
@app.route('/canvas', methods=['POST'])
def canvas():
    canvasdata = request.form['canvasimg']
    encoded_data = request.form['canvasimg'].split(',')[1]

    # Decode base64
    nparr = np.fromstring(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Convert 3 channel to 1 channel
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('280x280.jpg', gray_image)

    # Resize to (28, 28)
    gray_image = cv2.resize(gray_image, (28, 28), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite('28x28.jpg', gray_image)

    # Expand to (1, 28, 28)
    img = np.expand_dims(gray_image, axis=0)

    try:
        prediction = np.argmax(model.predict(img))
        print(f"Prediction Result : {str(prediction)}")
        return render_template('drawing.html', title=title, response=str(prediction), canvasdata=canvasdata, success=True)
    except Exception as e:
        return render_template('drawing.html', title=title, response=str(e), canvasdata=canvasdata)

## Creating the Upload File App
Right now, we are inside the foler **"templates"** in **"home.html"**
On this HTML page, the user can upload images (28x28) to test the program and receive the result on the screen. Furthermore, it is through this that, by clicking on the "Go to Drawing Page" button, you can test the app by drawing, using the touchpad, the digits to be tested.
In the root, there is a folder **"digits"** that has examples to test here.

![Home Html](htmlPages/HomeHTML.jpeg)

## Creating the Drawing App
Right now, we are inside the foler **"templates"** in **"drawing.html"**. We use HTML5 canvas to create a simple working drawing app which can be used to write digits using mouse. Then we can send into our server into base64 image data. As on the first page, here the user, after drawing the digit to be tested, receives the result of the analysis on the screen.

![Drawing Html](htmlPages/DrawingHTML.jpeg)

## Deployment
To test our app we need to run the file "wsgi.py" and open the server show

In [ ]:
from app.main import app

if __name__ == '__main__':
    app.run(debug = True)

We shoul see something like this
**Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)**

# Conlusion
In conclusion, this study delved into the application of an Artificial Neural Network (ANN) for the recognition of handwritten digits, leveraging the MNIST dataset. Throughout the project, we explored fundamental concepts of machine learning and neural networks, emphasizing the chosen architecture tailored to the task at hand.

The implementation and training of the ANN unveiled the model's ability to generalize complex patterns present in various samples of handwritten digits. The normalization of input data and the selected architecture proved to be crucial elements for the effective performance of the model.

However, we can also conclude that the accuracy of our app can be improved since, especially when uploading files, it fails several times. There are several reasons why this happens, which includes **Diversity in Training Data:** If the training dataset is not sufficiently representative in terms of diverse writing styles, sizes, and inclinations, the ANN may struggle to generalize to previously unseen cases.
**Overfitting:** Overfitting occurs when the ANN overly adjusts to specific training data and fails to generalize well to new data. This can happen if the model is too complex relative to the complexity of the data or if there is no validation data to monitor performance during training. We therefore conclude that it is a good ANN for recognizing handwritten digits but could be improved.